# Dice trading

## Part 4 : Simple game

##### 4.a)

$S_t = \{p_t, d_t\}$, avec $p_t$ le nombre de points au tour t et $d_t$ le nombre de dés au tour t

$a_t = \{0, 1\}$, soit acheter un nouveau dé, soit ne rien faire au tour t

$p_t \geq 0$

$0 d_t \leq 3$

$d_{t+1} = d_t + a_{d+1}$

$p_{t+1} = p_t - 5*a_t + \max_{i \in \{1, d_t\}} (u_i)$ ($u_i$ est le score du ième dé)

##### 4.b) Implement an heuristic

3 variables décrivent l'état du système S :
- Le total de point $p_{t}$. Ce total commençant à 0, et le gain maximum étant de 6 par tour, on en déduit l'encadrement suivant : $0 \leq p \leq 60$
- L'action d'acheter un dé $a_{t}$. On a $a_t \in {0,1}$
- Le tour $t$, qui est compris entre 1 et 10

On en déduit que toute stratégie peut être représentée par une matrice 3D de dimension $61*3*10$

REMARQUE IMPORTANTE POUR LA SUITE : La représentation d'une heuristique par une matrice n'est pas unique, car certains états sont inaccessibles (par exemple, avoir plus de 0 points au premier tour). Cela sera pertinent pour la question 4.g).

In [1]:
T=10
never_buy_heuristic = zeros(6*T + 1, 3, T)


buy_heuristic = ones(6*T + 1, 3, T)
for p in 1:6*T + 1
    for t in 1:T
        buy_heuristic[p, 3, t] = 0 # can't buy more than 3 dices
    end
end
for p in 1:6
    for t in 1:T
        for d in 1:3
            buy_heuristic[p, d, t] = 0 # can't buy dices with less than 6 points
        end
    end
end

In [2]:
#=  OBSOLETE CODE
function never_buy_heuristic(s,t) # Always returns "Don't Buy" (never buy dice) 
    return "Don't Buy"
end
=#

##### 4.c) Write a simulator giving $V_{\pi}$ in a confidence interval of 95%

In [3]:
using Pkg
Pkg.add("Distributions")
using Distributions
using Random

    Updating registry at `C:\Users\erwan\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\erwan\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\erwan\.julia\environments\v1.8\Manifest.toml`


In [4]:
#= OBSOLETE CODE
function simulator_simple_game(heuristic, history=false, T=10)
    nb_dice = 1
    total_points = 0
    println("test")
    if history
        history_list = []
    end

    for turn in 1:T
        action = heuristic((total_points, nb_dice), turn)
        @assert action != "Buy" || total_points >= 6 "RuleError You don't have enough points to buy a dice"
        if action == "Buy" && total_points >= 6 # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
            nb_dice += 1
            total_points -= 5
        end
        dice_roll = maximum(rand(1:6, nb_dice))
        total_points += dice_roll

        if history
            push!(history_list, (action, dice_roll, total_points))
        end
    end
    
    if history
        return total_points, history_list
    end
    return total_points
end
simulator_simple_game(never_buy_heuristic, true, 10)
=#

In [5]:
function simulator_simple_game(politic, T=10, N= 100)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    Liste = [] # List of the total points after T turns for each game
    for j in 1:N
        total_points = 0 # The player starts with 0 points
        nb_dice = 1 # The player starts with 1 dice
        for turn in 1:T
            action = politic[total_points + 1, nb_dice, turn] # Due to Julia's indexing system, we need to add 1 to the total_points
            @assert action != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice. State: $((total_points, nb_dice, turn))"
            @assert action != 1 || nb_dice < 3       "RuleError You can't have more than 3 dices. State: $((total_points, nb_dice, turn))"
            if action == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            dice_roll = maximum(rand(1:6, nb_dice))
            total_points += dice_roll
        end
        push!(Liste, total_points) 
    end
    return mean(Liste), [mean(Liste) - (1.96*std(Liste)/sqrt(N)), mean(Liste) + (1.96*std(Liste)/sqrt(N))]
end

T=10
simulator_simple_game(buy_heuristic, T, 10000)

(35.8213, [35.725217670474464, 35.91738232952554])

In [6]:
# function compute_expected_cost(heuristic, p=0.05):
#     #p is the error level
#     n=0 # Number of game simulated
#     emp_mean=0
#     emp_dev=0

#     d = Normal()  # Crée une distribution normale standard
#     p = 0.95  # Probabilité pour laquelle vous voulez trouver le quantile
#     q = quantile(d, p)  # Calcule le quantile
    
#     while n<1 and 
#     end
# end

##### 4.d)

On note $X$, $Y$, $Z$ les variables aléatoires associées aux trois dés. Par indépendance, on a :

$\forall k \in 1:6$

$P(Y=k)=1/6$

$P(max(X,Y)\leq k) = P((X\leq k) \cap (Y\leq k)) = P(X\leq k)*P(Y\leq k) = \frac{k^2}{36} \Rightarrow P(max(X,Y)=k) = P(max(X,Y)\leq k) - P(max(X,Y)\leq k-1) = \frac{2k-1}{36}$

$P(max(X,Y,Z)\leq k) = P((X\leq k) \cap (Y\leq k) \cap (Z\leq k)) = P(X\leq k)*P(Y\leq k)*P(Z\leq k) = \frac{k^3}{216} \Rightarrow P(max(X,Y,Z)=k) = P(max(X,Y,Z)\leq k) - P(max(X,Y,Z)\leq k-1) = \frac{3k^2-3k+1}{216}$

In [7]:
#= ??? Useless code ???
for i in 1:6
    println(1 * (i)^2 * 3 - 2)
end
=#

In [8]:
function law_max(nb_dices)
    @assert nb_dices > 0 "RuleError You must have at least one dice"
    @assert nb_dices <= 3 "RuleError You can't have more than 3 dices"
    
    if nb_dices == 1
        return DiscreteUniform(1, 6)
    end
    
    if nb_dices == 2
        values = 1:6
        probs = [1/36, 3/36, 5/36, 7/36, 9/36, 11/36]
        return Categorical(probs)
    end

    if nb_dices == 3
        values = 1:6
        probs = [1/216, 7/216, 19/216, 37/216, 61/216, 91/216]
        return Categorical(probs)
    end
end

function general_law_max(nb_dices) #useful for 4.h) and after
    @assert nb_dices > 0 "RuleError You must have at least one dice"
    if nb_dices == 1
        return DiscreteUniform(1, 6)
    end
    values = 1:6
    probs = [(k/6)^nb_dices - ((k-1)/6)^nb_dices for k in values]
    return Categorical(probs)
end

general_law_max (generic function with 1 method)

##### 4.e)

In [9]:
#= Outdated code ???
Q = zeros(61, 3, 10, 2) #60 points, 3 dices, 10 turns, 2 actions

#Construct the value function for each state at each time
V = zeros(61, 3, 10)

P = zeros(61, 3, 10, 2, 61) #60 points, 3 dices, 10 turns, 2 actions, 60 points 

for p in 0:60
    for d in 1:3
        for t in 1:10
            for a in 0:1
                if d + a <= 3
                    law = law_max(d + a)
                    for p_prime in 0:60
                        P[p + 1, d, t, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                    end
                end
            end
        end
    end
end
policy_markov = zeros(61, 3, 10)

println(law_max(2)[5])
println(pdf(law_max(2), 5))
=#

We now proceed to the dynamic programming

In [10]:
function dp(T)
    Q = zeros(T*6 + 1, 3, T, 2) # Action-value function Q(s,a), giving the estimates the expected cumulative future rewards starting from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions)
    #Construct the value function for each state at each time
    V = zeros(T*6 + 1, 3, T+1) # Value function V(s), giving the expected cumulative future rewards starting from state s (60 points, 3 dices, 10 turns + 1 for the final state)

    P = zeros(T*6 + 1, 3,  2, T*6 + 1) # Transition probability matrix P(s'|s,a), giving the probability of transitioning to state s' from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions, 60 points)
    # Note that s = (p,d,t), but the probability doesn't depend on t, so we can simplify the matrix to P(p'|p,d,a)
    for p in 0:T*6
        for d in 1:3
            for a in 0:1
                if d + a <= 3
                    law = law_max(d + a) # law_max returns the distribution of the maximum of d + a dices, pdf(law, k) gives the probability of rolling k
                    for p_prime in 0:T*6
                        P[p + 1, d, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                    end
                end
            end
        end
    end
    policy_markov = zeros(T * 6 + 1, 3, T)
    for t in T:-1:1
        for p in 0:T * 6
            for d in 1:3
                V[p + 1, d, t] = -100000 # We initialize the value function to a very low value
                for a in 0:1
                    Q[p + 1, d, t, a + 1] = 0
                    if d + a <= 3 && a * (p-6) >= 0        # We assure that we can't have more than 3 dices AND that we can't buy a dice if we don't have enough points
                        for p_prime in 0:T * 6
                            transition_cost = p_prime - p
                            Q[p + 1, d, t, a + 1] += P[p + 1, d, a + 1, p_prime + 1] * (transition_cost + V[p_prime + 1, d + a, t+1])
                        end
                        if Q[p + 1, d, t, a + 1] > V[p + 1, d, t]
                            V[p + 1, d, t] = Q[p + 1, d, t, a + 1]
                            policy_markov[p + 1, d, t] = a
                        end
                    end
                end
            end
        end
    end
    policy_markov = round.(Int, policy_markov)
    return policy_markov
end

dp (generic function with 1 method)

### La valeur optimale trouvée par programmation dynamique est la suivante

In [11]:
T=10
policy_markov = dp(T)
C = simulator_simple_game(policy_markov, T, 10000)

(37.7146, [37.61607129733636, 37.81312870266363])

 #### On est content car elle est mieux que la valeur de l'heuristique

##### 4.g)
Comme indiqué en 4.b), la stratégie "ne jamais acheté" peut être représentée par autre chose qu'une matrice nulle (à cause des valeurs inaccesibles). Ces valeurs inaccesibles sont  :

- $\pi [p,1,t]$ avec $p>6(t-1)$ ou $p<t-1$

- $\pi [p,2,t]$ avec $t=1$ ou $p>6(t-1) - 5$ ou $p<t-1-5$

- $\pi [p,3,t]$ avec $t<=2$ ou $p>6(t-1) - 10$ ou $p<t-1-10$

Qu'on peut résumer en :

- $\pi [p,d,t]$ avec $d+1>t$ ou $p>6(t-1) - 5*(d-1)$ ou $p<t - 1 - 5(d-1)$

(On peut remarquer que la formule précédente marche aussi pour $d$ différent de 3)

Pour résoudre ce problème, la fonction suivante mettra toutes ces valeurs inaccessible à 0.


In [12]:
function nullify_inaccessible_values(matrice_3d)
    for p in 1:size(matrice_3d, 1)
        for d in 1:size(matrice_3d, 2)
            for t in 1:size(matrice_3d, 3)
                if d+1>t || p-1 > 6*(t-1) - 5*(d-1) || p - 1 < t - 1 - 5*(d - 1) # don't forget that Julia's indexing system starts at 1
                    matrice_3d[p, d, t] = 0
                end
            end
        end
    end
    return matrice_3d
end

nullify_inaccessible_values (generic function with 1 method)

In [13]:
# We will now implement the dynamic programming with descending time horizon to see when it is never intersting to buy a dice
function is_null(matrice_3d)
    for i in 1:size(matrice_3d, 1)
        for j in 1:size(matrice_3d, 2)
            for k in 1:size(matrice_3d, 3)
                if matrice_3d[i, j, k] != 0
                    return false
                end
            end
        end
    end
    return true
end

for T in 10:-1:1
    policy_markov = nullify_inaccessible_values(dp(T))
    if is_null(policy_markov)
        println("The optimal policy is to never buy a dice after $T turns")
        break
    end
end

The optimal policy is to never buy a dice after 6 turns


In [15]:
function simple_test_strategy(T, π, d=1)
    for t in 1:T
        println("t=$t : ", π[:, d, t])
    end
end
simple_test_strategy(20, nullify_inaccessible_values(dp(20)))

t=1 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=2 : [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=3 : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### 4.h)
On reprend les fonctions précédentes en enlevant la condition d<3 :

In [16]:
function no_dice_limit_simulator(politic, T=10, N= 100)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    Liste = [] # List of the total points after T turns for each game
    for j in 1:N
        total_points = 0 # The player starts with 0 points
        nb_dice = 1 # The player starts with 1 dice
        for turn in 1:T
            action = politic[total_points + 1, nb_dice, turn] # Due to Julia's indexing system, we need to add 1 to the total_points
            @assert action != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice. State: $((total_points, nb_dice, turn))"
            if action == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            dice_roll = maximum(rand(1:6, nb_dice))
            total_points += dice_roll
        end
        push!(Liste, total_points) 
    end
    return mean(Liste), [mean(Liste) - (1.96*std(Liste)/sqrt(N)), mean(Liste) + (1.96*std(Liste)/sqrt(N))]
end

function no_dice_limit_dp(T)
    Q = zeros(T*6 + 1, T+1, T, 2) # Action-value function Q(s,a), giving the estimates the expected cumulative future rewards starting from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions)
    #Construct the value function for each state at each time
    V = zeros(T*6 + 1, T+1, T+1) # Value function V(s), giving the expected cumulative future rewards starting from state s (60 points, 3 dices, 10 turns + 1 for the final state)

    P = zeros(T*6 + 1, T+1,  2, T*6 + 1) # Transition probability matrix P(s'|s,a), giving the probability of transitioning to state s' from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions, 60 points)
    # Note that s = (p,d,t), but the probability doesn't depend on t nor d', so we can simplify the matrix to P(p'|p,d,a)
    for p in 0:T*6
        for d in 1:T
            for a in 0:1
                law = general_law_max(d + a) # law_max returns the distribution of the maximum of d + a dices, pdf(law, k) gives the probability of rolling k
                for p_prime in 0:T*6
                    P[p + 1, d, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                end
            end
        end
    end
    policy_markov = zeros(T * 6 + 1, T+1, T)
    for t in T:-1:1
        for p in 0:T * 6
            for d in 1:T
                V[p + 1, d, t] = -100000 # We initialize the value function to a very low value
                for a in 0:1
                    Q[p + 1, d, t, a + 1] = 0
                    if  a * (p-6) >= 0        # We assure that we can't buy a dice if we don't have enough points
                        for p_prime in 0:T * 6
                            transition_cost = p_prime - p
                            Q[p + 1, d, t, a + 1] += P[p + 1, d, a + 1, p_prime + 1] * (transition_cost + V[p_prime + 1, d + a, t+1])
                        end
                        if Q[p + 1, d, t, a + 1] > V[p + 1, d, t]
                            V[p + 1, d, t] = Q[p + 1, d, t, a + 1]
                            policy_markov[p + 1, d, t] = a
                        end
                    end
                end
            end
        end
    end
    policy_markov = round.(Int, policy_markov)
    return policy_markov
end

no_dice_limit_dp (generic function with 1 method)

Remarquons que dans le cas $T=10$, pas plus qu'un dé n'était acheté. Considérons le cas $T=50$

In [17]:
simple_test_strategy(50, nullify_inaccessible_values(no_dice_limit_dp(50)), 5)

t=1 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=2 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

On remarque qu'il reste interessant d'acheter des dés au delà du 5ème dé dans certaines situations.

## Part 5 : Spending dices

### 5.1)

Remarquons que ce nouveau jeu est très différent du précédent : en effet, le joueur doit effectué une action avant et après le lancer. Il n'est donc pas possible de simplement "complexifier" le modèle précédent en ajoutant une action, il faut modifier entièrement les états. Pour celà, on découpe chaque tour en deux parties : avant le lancer et après le lancer. On pourrait doubler le pas de temps t, mais afin de garder la cohérence entre le code et les données de l'énoncé, nous allons introduire la variable binaire r et l'entier vr.

On définit alors un état $s = (p, d, t, r)$ avec :
- $p_{t,r}$ le nombre de points du joueur au début du tour t
- $d_{t,r}$ le nombre de dés au début du tour t
- $vr_{t,r}$ est la valeur du lancer au tour t. Notons que $vr_{t,r}$ n'est pas déterminé pour r=0, on ignorera donc sa valeur.
- $t$ le numéro du tour
- $r$ indicatrice de si les dés du tour t ont déjà été lancés

L'espace des actions change selon la valeur de $r_t$ :
- Si $r$ = 0 (dés non lancés), $a_{t,r}$ = 1 si un dé est acheté, 0 sinon
- Si $r$ > 1 (dés lancés), $a_{t,r}$ =1 si le joueur vend un dé pour doubler son score

Pour une partie à T tours avec une limite de k dés, les contraintes sont les suivantes :

- $\forall (t,r), \quad 0 \leq p_{t,r} \leq 7t \quad$ (le score ne peut être de plus de 7 au global car au mieux on double le score du dé et on paye pour le nouveau dé à chaque tour)
- $\forall (t,r), \quad 0 \leq d_{t,r} \leq k \quad$ (pour cette question, k = 5) 

(Notons qu'on déjà attribué les points une fois les dés lancés, et les rajouter une deuxième fois dans le cas de la vente d'un dé)
- $vr_{t,1} = \max_{i \in \{1, d_{t,0}+a_{t,0}\}} (u_i) \quad$ ($u_i$ est le score du ième dé) 
- $p_{t,1} = p_{t,0} - 5*a_{t+1,0} + vr_{t,1}$
- $p_{t+1,0} = p_{t,1} + a_{t,1}*(vr_{t,1}) $ 
- $d_{t,1} = d_{t,0} + a_{t,0}$
- $d_{t+1, 0} = d_{t,1} - a_{t,1} $

Avec notre modélisation du problème, une stratégie peut être représentée par une matrice 4D de taille $9T$ x $k$ x $6$ x $T$ x $2$ (l'unicité de la matrice pour une stratégie n'est toujours pas garantie)

In [18]:
function game2_simulator(strategy, T=10, k=5, N=10000)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    Liste = [] # List of the total points after T turns for each game
    for j in 1:N
        total_points = 0 # The player starts with 0 points
        nb_dice = 1 # The player starts with 1 dice
        for turn in 1:T
            # Part 1, before the dice roll
            r=0
            vr=1 # vr is undefined at this points, so we will use 1 as a placeholder
            action = strategy[total_points + 1, nb_dice, vr, turn, r+1] # First action, taken before the dice roll
            @assert action != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice. State: $((total_points, nb_dice, vr, turn, r))"
            @assert action != 1 || nb_dice < k       "RuleError You can't have more than k dices. State: $((total_points, nb_dice, vr, turn, r))"
            if action == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            vr = maximum(rand(1:6, nb_dice))
            total_points += vr

            # Part 2, after the dice roll
            r=1
            action = strategy[total_points + 1, nb_dice, vr, turn, r+1] # Second action, taken after the dice roll
            @assert action != 1 || nb_dice > 1 "RuleError You can't sell your unique die. State: $((total_points, nb_dice, turn))"
            if action == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice -= 1
                total_points += vr
            end
        end
        push!(Liste, total_points) 
    end
    return mean(Liste), [mean(Liste) - (1.96*std(Liste)/sqrt(N)), mean(Liste) + (1.96*std(Liste)/sqrt(N))]
end

game2_simulator (generic function with 4 methods)

In [43]:
T=50
k=5
never_buy_nor_sell_heuristic = zeros(7*T + 1, k, 6, T,2)

351×5×6×50×2 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 ⋮                   
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 ⋮                   
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0

In [48]:
game2_simulator(never_buy_nor_sell_heuristic, T, k, 10000)

(175.0408, [174.80468576469616, 175.27691423530382])

On remarque que notre simulateur donne des résultats similaires au précédent pour la stratégie n'achetant rien.

In [56]:
function game2_dp(T,k=5)
    Q = zeros(T*7 + 1, k, 6, T, 2, 2) # Action-value function Q(s,a), giving the estimates the expected cumulative future rewards starting from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions)
    #Construct the value function for each state at each time
    V = zeros(T*7 + 1, k, 6, T+1, 2) # Value function V(s), giving the expected cumulative future rewards starting from state s (60 points, 3 dices, 10 turns + 1 for the final state)

    P = zeros(T*7 + 1, k,  2, T*7 + 1) # Transition probability matrix P(s'|s,a), giving the probability of transitioning to state s' from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions, 60 points)
    #= Note that s = (p,d,t), but the probability doesn't depend on d', vr, vr', and the transition from (t,1) to (t+1,0) is deterministic,
       so we can simplify the matrix to P(p'|p,d,a) and consider it only to compute transition from (t,0) to (t,1). 
    =#
    for p in 0:T*7
        for d in 1:k
            for a in 0:1
                if d + a <= k
                    law = general_law_max(d + a) # law_max returns the distribution of the maximum of d + a dices, pdf(law, k) gives the probability of rolling k
                    for p_prime in 0:T*7
                        P[p + 1, d, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                    end
                end
            end
        end
    end
    # Note that we have p' = p - 5a + vr
    policy_markov = zeros(T * 7 + 1, k, 6, T, 2)
    for t in T:-1:1
        for r in 1:-1:0
            for p in 0:T*7
                for d in 1:k
                    for vr in 1:6
                        V[p + 1, d, vr, t, r + 1] = -100000 # We initialize the value function to a very low value
                        for a in 0:1
                            Q[p + 1, d, vr, t, r + 1, a + 1] = 0 #Useless ?
                            if r == 0 # We are in the first part of the turn, before the dice roll
                                if  a * (p-6) >= 0 && d+a <= k     # We assure that we can't buy a dice if we don't have enough points AND that we can't have more than k dices
                                    for p_prime in 0:T * 6
                                        transition_cost = p_prime - p
                                        if transition_cost + 5*a >= 1 && transition_cost + 5*a <= 6 # We assure that vr is between 1 and 6
                                            Q[p + 1, d, vr, t, r + 1, a + 1] += P[p + 1, d, a + 1, p_prime + 1] * (transition_cost + V[p_prime + 1, d + a, transition_cost + 5*a, t, r+2]) # We add 2 to r to go to the second part of the turn
                                        end
                                    end
                                    #if p>5
                                    #    println("p=$p, d=$d, vr=$vr, t=$t, r=$r, a=$a", " Q=", Q[p + 1, d, vr, t, r + 1, a + 1], " V=", V[p + 1, d, vr, t, r + 1])
                                    #end
                                    if Q[p + 1, d, vr, t, r + 1, a + 1] > V[p + 1, d, vr, t, r + 1]
                                        V[p + 1, d, vr, t, r + 1] = Q[p + 1, d, vr, t, r + 1, a + 1]
                                        policy_markov[p + 1, d, vr, t, r + 1] = a
                                    end
                                end
                            end
                            if r == 1 # We are in the second part of the turn, after the dice roll
                                #in this case, V is deterministic with vr and a 
                                if a == 0 && V[p + 1, d, vr, t, 2] < V[p + 1, d, vr, t+1, 1] # We don't sell the dice, so V don't change
                                    V[p + 1, d, vr, t, 2] = V[p + 1, d, vr, t+1, 1]
                                    policy_markov[p + 1, d, vr, t, r + 1] = a
                                end
                                if a == 1 && d>1 && V[p + 1, d, vr, t, 2] < vr + V[p + 1, d-1, vr, t+1, 1]  # We sell the dice (if it's possible), so V[p,d,vr,t,r] = vr + the expected value with one less dice
                                    V[p + 1, d, vr, t, 2] = vr + V[p + 1, d-1, vr, t+1, 1]
                                    policy_markov[p + 1, d, vr, t, r + 1] = a
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    policy_markov = round.(Int, policy_markov)
    return policy_markov
end

game2_dp (generic function with 2 methods)

In [63]:
function simple_test_strategy(T, π, d=1, vr=1, r=0)
    for t in 1:T
        println("t=$t : ", π[:, d, vr, t, r+1])
    end
end
k=5
T=50
simple_test_strategy(T, game2_dp(T, k), 2, 1, 1)

In [60]:
T=50
k=5
game2_simulator(game2_dp(T, k), T, k, 10000)

(288.1211, [287.92706273658166, 288.31513726341836])

(WIP : expliquer la stratégie)

5.2)

In [62]:
T=50
k=T
game2_simulator(game2_dp(T, k), T, k, 10000)

(305.3232, [305.20026803186556, 305.4461319681344])

Comme attendu, le prix augmente 

# PLEASE IGNORE WHAT IS AFTER THIS

### 5.1)

Remarquons que ce nouveau jeu est très différent du précédent : en effet, le joueur doit effectué une action avant et après le lancer. Il n'est donc pas possible de simplement "complexifier" le modèle précédent en ajoutant une action, il faut modifier entièrement les états.

On définit alors un état $s = (p, d, r, t)$ avec :
- $p_t$ le nombre de points du joueur au début du tour t
- $d_t$ le nombre de dés au début du tour t
- $r_t$ le résultat du lancer au tour t (si les dés n'ont pas encore été lancé, on prendra $r_t=0$)
- $t$ le numéro du tour

L'espace des actions change selon la valeur de $r_t$ :
- Si $r_t$ = 0 (dés non lancés), $a_{t,r}$ = 1 si un dé est acheté, 0 sinon
- Si $r_t$ > 1 (dés lancés), $a_{t,r}$ =1 si le joueur vend un dé pour doubler son score

Pour une partie à T tours avec une limite de k dés, les contraintes sont les suivantes :

- $9t \geq p_t \geq 0$ (le score ne peut être doublé d'un jour sur 2 car il faut acheter les dés)
- $0 \leq d_t \leq k$ (pour cette question, k = 5) 
- $r_t = \max_{i \in \{1, d_t+a_{t,0}\}} (u_i)$ ($u_i$ est le score du ième dé)
- $t = t + 1_{r_t > 0}$
- $d_{t+1} = d_t + a_{t+1,0} - a_{t+1,r}$ 
- $p_{t+1} = p_{t} - 5*a_{t+1,0} + (1+a_{t,r}) * r_t$ 

Avec notre modélisation du problème, une stratégie peut être représentée par une matrice 4D de taille $9T$ x $k$ x $7$ x $T$ (l'unicité de la matrice pour une stratégie n'est toujours pas garantie)

In [45]:
function spending_game_simulator(strategy, T=10, k=5, N=100)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    Liste = [] # List of the total points after T turns for each game
    for j in 1:N
        total_points = 0 # The player starts with 0 points
        nb_dice = 1 # The player starts with 1 dice
        for turn in 1:T
            r = 0  # The score of the dice roll
            action_0 = strategy[total_points + 1, r, turn] # First action, taken before the dice roll
            @assert action_0 != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice. State: $((total_points, nb_dice, r, turn))"
            @assert action_0 != 1 || nb_dice < k       "RuleError You can't have more than k dices. State: $((total_points, nb_dice, r, turn))"
            r = maximum(rand(1:6, nb_dice + action_0))
            action_r = strategy[total_points + 1, r, turn] # Second action, taken after the dice roll
            @assert action_r != 1 || nb_dice + action_0 > 1 "RuleError You can't sell your unique die. State: $((total_points, nb_dice, r, turn), action_0)"
            if action_r == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
        end
        push!(Liste, total_points) 
    end
    return mean(Liste), [mean(Liste) - (1.96*std(Liste)/sqrt(N)), mean(Liste) + (1.96*std(Liste)/sqrt(N))]
    
end

ErrorException: syntax: invalid interpolation syntax

# Code obsolète

Redéfinissons $a_{t}$ tel que : 
- $a_{t}$ = 1 : achât d'un dé
- $a_{t}$ = 0 : pas d'achât
- $a_{t}$ = -1 : vente d'un dé

On a alors les conditions : 
- $p_t \geq 0$
- $0 d_t \leq k$ (pour cette question, k = 5) 
- $d_{t+1} = d_t + a_{d+1}$
- $p_{t+1} = p_t - 5*max(a_t,0) + (1-min(0,a_t))*\max_{i \in \{1, d_t\}} (u_i)$ ($u_i$ est le score du ième dé)

Remarquons que $p_t$ n'est plus borné par $6t$, mais l'est toujours par $9t$ (puisqu'il est impossible)
On recrée le simulateur :

In [46]:
function spending_simulator(politic, T=10, N= 100, k=5)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    Liste = [] # List of the total points after T turns for each game
    for j in 1:N
        total_points = 0 # The player starts with 0 points
        nb_dice = 1 # The player starts with 1 dice
        for turn in 1:T
            action = politic[total_points + 1, nb_dice, turn] # Due to Julia's indexing system, we need to add 1 to the total_points
            @assert action != 1 || nb_dice < k       "RuleError You can't have more than k dices. State: $((total_points, nb_dice, turn))"
            @assert action != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice. State: $((total_points, nb_dice, turn))"
            if action == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            if action == -1
                nb_dice -= 1
            end
            dice_roll = maximum(rand(1:6, nb_dice))
            total_points += Int((1-min(0,action))*dice_roll)
        end
        push!(Liste, total_points) 
    end
    return mean(Liste), [mean(Liste) - (1.96*std(Liste)/sqrt(N)), mean(Liste) + (1.96*std(Liste)/sqrt(N))]
end

function test_spending_sim(T=10, N= 100, k=5)
    never_buy_heuristic = zeros(6*T + 1, k, T)
    println("Never buy heuristic: ", spending_simulator(never_buy_heuristic, T, N, k))
end

test_spending_sim()

In [47]:
function spending_dp(T,k=5)
    Q = zeros(T*6 + 1, k, 6, T, 2) # Action-value function Q(s,a), giving the estimates the expected cumulative future rewards starting from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions)
    #Construct the value function for each state at each time
    V = zeros(T*6 + 1, k, 6, T+1) # Value function V(s), giving the expected cumulative future rewards starting from state s 

    P = zeros(T*6 + 1, k, 2, T*6 + 1) # Transition probability matrix P(s'|s,a), giving the probability of transitioning to state s' from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions, 60 points)
    # Note that s = (p,d,t), but the probability doesn't depend on t nor d, so we can simplify the matrix to P(p'|p,a)
    for p in 0:T*6
        for d in 1:k
            for a in 0:1
                if d + a <= k
                    law = general_law_max(d + a) # law_max returns the distribution of the maximum of d + a dices, pdf(law, k) gives the probability of rolling k
                    for p_prime in 0:T*6
                        
                        P[p + 1, d, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * max(a,0))
                    end
                end
            end
        end
    end
    policy_markov = zeros(T * 6 + 1, k, T)
    for t in T:-1:1
        for p in 0:T * 6
            for d in 1:k
                for vr in 1:6
                    V[p + 1, d, t] = -100000 # We initialize the value function to a very low value
                    for a in 0:1
                        Q[p + 1, d, t, a + 1] = 0
                        if d + a <= k && a * (p-6) >= 0        # We assure that we can't have more than 3 dices AND that we can't buy a dice if we don't have enough points
                            for p_prime in 0:T * 6
                                transition_cost = p_prime - p
                                Q[p + 1, d, t, a + 1] += P[p + 1, d, a + 1, p_prime + 1] * (transition_cost + V[p_prime + 1, d + a, t+1])
                            end
                            if Q[p + 1, d, t, a + 1] > V[p + 1, d, t]
                                V[p + 1, d, t] = Q[p + 1, d, t, a + 1]
                                policy_markov[p + 1, d, t] = a
                            end
                        end
                    end
                end
            end
        end
    end
    policy_markov = round.(Int, policy_markov)
    return policy_markov
end

spending_dp (generic function with 2 methods)